In [1]:
"""
# Chapter 10 Data Aggregation and Group Operations.
Categorizing a dataset and applying a function to each group, whether an aggreation or transformation,
is a critical componet of a data analysis workflow. 

# GroupBy Mechanics: - Split - Apply - Combine -
In the first stage of the process:
data contained in a pandas object is spit into groups based on one or more keys that you provide.
The splittting is performed on a particular axis of an object,
then after a function is applied to each group, producing a new value.
Resulting object kind if dependent of mutation to data.
"""

%matplotlib notebook
# Financial Time Series 
import numpy as np 
import pandas as pd
from pylab import mpl, plt
plt.style.use('seaborn')
mpl.rcParams['font.family'] = 'serif'
%matplotlib inline
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', 2000)
pd.set_option('display.width', 1000)

In [2]:
df  = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                    'key2' : ['One', 'two', 'one', 'two', 'one'],
                    'data1' : np.random.randn(5),
                    'data2' : np.random.randn(5)
                })
display(df)

# TO compute the mean of the data1 column suing the labels from key1. 
grouped = df['data1'].groupby(df['key1'])

display(grouped.std())

means = df['data1'].groupby([df['key1'], df['key2']]).mean()

# Here we grouped the data using two keys, and the resulting series now has a hierarchial index consisting of the unique of keys observed
  
display(means)

display(means.unstack())

# Regardless of the objective in using groupby, a generally useful GroupBy method is size, which reurns a Series containing group sizes:

display(df.groupby(['key1', 'key2']).size())

,key1,key2,data1,data2
0,a,One,-0.065630,0.498820
1,a,two,1.058542,1.050469
2,b,one,0.510699,1.405132
3,b,two,0.203782,-2.313964
4,a,one,-0.165070,0.002633


key1
a    0.679568
b    0.217023
Name: data1, dtype: float64

key1  key2
a     One    -0.065630
      one    -0.165070
      two     1.058542
b     one     0.510699
      two     0.203782
Name: data1, dtype: float64

key2,One,one,two
key1,,,
a,-0.06563,-0.165070,1.058542
b,NaN,0.510699,0.203782


key1  key2
a     One     1
      one     1
      two     1
b     one     1
      two     1
dtype: int64

In [3]:
"""
# Iterating Over Groups: The GroupBy object supports iteration,
genarating a sequence of 2-tuples containing  the group  name along with the chuck of data. 
"""

for name, group in df.groupby('key1'):
    print(name)
    print(group)
    
        
# In the case of multiple keys, the first element in the tuple of key values:

for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)
    
#  Of course,you can choose to do whaetever with thee pieces of data.
# Computing a dict of the data pieces as a one liner:

pieces = dict(list(df.groupby('key1')))

display(pieces['b'])

# By default groupby groups on axis =0, but you can hroup on any of the other axes.

a
  key1 key2     data1     data2
0    a  One -0.065630  0.498820
1    a  two  1.058542  1.050469
4    a  one -0.165070  0.002633
b
  key1 key2     data1     data2
2    b  one  0.510699  1.405132
3    b  two  0.203782 -2.313964
('a', 'One')
  key1 key2    data1    data2
0    a  One -0.06563  0.49882
('a', 'one')
  key1 key2    data1     data2
4    a  one -0.16507  0.002633
('a', 'two')
  key1 key2     data1     data2
1    a  two  1.058542  1.050469
('b', 'one')
  key1 key2     data1     data2
2    b  one  0.510699  1.405132
('b', 'two')
  key1 key2     data1     data2
3    b  two  0.203782 -2.313964


,key1,key2,data1,data2
2,b,one,0.510699,1.405132
3,b,two,0.203782,-2.313964


In [4]:
"""
# Selecting a Column or Subset of Columns:
It may be desirable to aggregate only a fre columns. 

To compute means for just the data2 column and get the reslut as a df 
"""

s_grouped = df.groupby(['key1', 'key2'])['data2']

display(s_grouped.mean())


key1  key2
a     One     0.498820
      one     0.002633
      two     1.050469
b     one     1.405132
      two    -2.313964
Name: data2, dtype: float64

In [5]:
"""
# Grouping with Dicts andf Series

Grouping information may exist in a form other than an array. 

"""
people = pd.DataFrame(np.random.randn(5, 5),
                     columns=['a', 'b', 'c', 'd', 'e'],
                     index=['Joe', 'Steven', 'Wes', 'Jim', 'Travis'])

people.iloc[2:3, [1,2]] = np.nan

display(people)

# Suppose a group corrspondence for the columns and want to sum togehter the clumns by group

mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
          'd': 'blue', 'e': 'red', 'f': 'orange'}

by_column = people.groupby(mapping, axis=1)

display(by_column.sum())

# The same fucntionality holds for Serues, which can be viewd as a fixed size-mapping

map_serries = pd.Series(mapping)

display(map_serries)

display(people.groupby(map_serries, axis=1).count())

,a,b,c,d,e
Joe,0.167194,1.581351,0.791578,-0.021579,0.114226
Steven,0.170596,-0.600523,0.547629,-1.199509,-1.626766
Wes,0.910573,NaN,NaN,0.473430,-1.067075
Jim,-1.512150,-0.127276,-0.278954,-0.933156,-0.627805
Travis,-0.551714,2.132527,1.335330,-0.891192,-0.185612


,blue,red
Joe,0.769999,1.862770
Steven,-0.651880,-2.056693
Wes,0.473430,-0.156501
Jim,-1.212110,-2.267231
Travis,0.444138,1.395201


a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

,blue,red
Joe,2,3
Steven,2,3
Wes,1,2
Jim,2,3
Travis,2,3


In [6]:
"""
# Grouping with Functions

Using Python finctions is a more generic way
of defiening a group mapping campared with a dict or Series.
Any functions passed as a group key will be called once per index value, with the retun being
used as the group names.


To group by the length of the names; while you could compute an array of string lengths,
it's simpler to just pass the len function
"""

display(people.groupby(len).sum())

# Mixing functions with arrays, dicts, or Series is not a problem
# as everything gets converted to arrays intenally 

key_list = ['one', 'one', 'one', 'two','two']

display(people.groupby([len, key_list]).min())

,a,b,c,d,e
3,-0.434383,1.454074,0.512623,-0.481305,-1.580654
6,-0.381117,1.532004,1.882959,-2.090701,-1.812378


a         b         c         d         e
3 one  0.167194  1.581351  0.791578 -0.021579 -1.067075
  two -1.512150 -0.127276 -0.278954 -0.933156 -0.627805
6 one  0.170596 -0.600523  0.547629 -1.199509 -1.626766
  two -0.551714  2.132527  1.335330 -0.891192 -0.185612

In [7]:
"""
# Data Agregations: Any data transformation that produces scalar values from arrays.
Many common aggregations, such as those found in page 302 have optimized implementations.
"""

display(df)

grouped = df.groupby('key1')

display(grouped['data1'].quantile(0.9))

# To use your own aggregation fucntions, pass any fucntion that
# agregates an arrat to the aggregate or agg method:

def peak_to_peak(arr):
    return arr.max() - arr.min()

display(grouped.agg(peak_to_peak))

display(grouped.describe())

,key1,key2,data1,data2
0,a,One,-0.065630,0.498820
1,a,two,1.058542,1.050469
2,b,one,0.510699,1.405132
3,b,two,0.203782,-2.313964
4,a,one,-0.165070,0.002633


key1
a    0.833708
b    0.480007
Name: data1, dtype: float64

C:\Users\JBarr\anaconda3\envs\dev\lib\site-packages\pandas\core\groupby\generic.py:303: FutureWarning: Dropping invalid columns in SeriesGroupBy.agg is deprecated. In a future version, a TypeError will be raised. Before calling .agg, select only columns which should be valid for the aggregating function.
  results[key] = self.aggregate(func)


,data1,data2
key1,,
a,1.223612,1.047835
b,0.306917,3.719096


data1                                                                      data2                                                                      
     count      mean       std       min       25%      50%       75%       max count      mean       std       min       25%       50%       75%       max
key1                                                                                                                                                       
a      3.0  0.275947  0.679568 -0.165070 -0.115350 -0.06563  0.496456  1.058542   3.0  0.517307  0.524162  0.002633  0.250727  0.498820  0.774644  1.050469
b      2.0  0.357240  0.217023  0.203782  0.280511  0.35724  0.433970  0.510699   2.0 -0.454416  2.629798 -2.313964 -1.384190 -0.454416  0.475358  1.405132

In [8]:
"""
# Column-Wise Multiple Funtion Application
# adding a tip pct columns
"""

tips = pd.read_csv('tips.csv')

tips['tip_pct'] = tips['tip'] / tips['total_bill']

display(tips)

# To aggregate using a different fucntion depending on hte column, or multiple fucntions at once.

grouped = tips.groupby(['day', 'smoker'])


,total_bill,tip,sex,smoker,day,time,size,tip_pct
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808
5,25.29,4.71,Male,No,Sun,Dinner,4,0.186240
6,8.77,2.00,Male,No,Sun,Dinner,2,0.228050
7,26.88,3.12,Male,No,Sun,Dinner,4,0.116071
8,15.04,1.96,Male,No,Sun,Dinner,2,0.130319
9,14.78,3.23,Male,No,Sun,Dinner,2,0.218539


In [9]:
# For descriptive statistics like those in table 10-1 use fucntion name as string

grouped_pct = grouped['tip_pct']

display(grouped_pct.agg('mean'))

# passing a list of fucntions or function names instead,
# u get bacj a DataFrame with column namess taken from the functions:

display(grouped_pct.agg(['mean', 'std', peak_to_peak]))

# Passing a list of (name, fucntion) tuples, 
# the first element of each tuple will be used as the DataFrame column names 
# (Think of a list of 2 tuples as an ordered mapping)

display(grouped_pct.agg([('foo', 'mean'), ('bar', np.std)]))

# With D DataFrame You have more options,specify a list of functions 
# to apply to all of the columns or different functions per column.
# Suppose we wanted to compute the same three statistics for the tip_pct and total bill

fucntions = ['count', 'mean', 'max']

result = grouped['tip_pct', 'total_bill'].agg(fucntions)

display(result)

display(result['tip_pct'])

# As before, a list of tuples with custom names can be passed

ftuples = [('Durchschnitt', 'mean'), ('Abeweichung', np.var)]

display(grouped['tip_pct', 'total_bill'].agg(ftuples))

# To apply potentially different functions to one or more of the columns.
# To do this, pass a dict to agg that contains a mapping of column names to any of the function 
# specification listed so far:

display(grouped.agg({'tip' : np.max, 'size' : 'sum'}))

display(grouped.agg({'tip_pct' : ['min', 'max', 'mean', 'std'],
                     'size': 'sum'}))

# Returning Aggregated Data Without Row Indexs: by passing as_index = false to groupby 

display(tips.groupby(['day','smoker'], as_index=False).mean())

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

C:\Users\JBarr\anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:24: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

...

C:\Users\JBarr\anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:34: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


tip_pct               total_bill            
            Durchschnitt Abeweichung Durchschnitt Abeweichung
day  smoker                                                  
Fri  No         0.151650    0.000791    18.420000   25.596333
     Yes        0.174783    0.002631    16.813333   82.562438
Sat  No         0.158048    0.001581    19.661778   79.908965
     Yes        0.147906    0.003767    21.276667  101.387535
Sun  No         0.160113    0.001793    20.506667   66.099980
     Yes        0.187250    0.023757    24.120000  109.046044
Thur No         0.160298    0.001503    17.113111   59.625081
     Yes        0.163863    0.001551    19.190588   69.808518

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


In [10]:
"""
# Apply: General split-apply-combine
The most general-purpose groupby method is apply.
apply splits the object being manipulated into pieces, invokes the passed fucntion on each piece,
and then attempts to concatenate the pieces together
"""

def top(df, n=5, columns='tip_pct'):
    return df.sort_values(by=columns)[-n:]

display(top(tips, n=6))

# Now, if we group by smoker, say and call apply with this function. we get the following 
# The top is called on each row group from the df and then the results are glued togehter abs
# using pandas.concat labeling the pieces with the group names
# The result therefore has hieratchial index, whose inner level contains index values from org df.

display(tips.groupby('smoker').apply(top))

display(tips.groupby(['smoker', 'day']).apply(top, n=1, columns='total_bill'))


,total_bill,tip,sex,smoker,day,time,size,tip_pct
109,14.31,4.00,Female,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Male,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,Male,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Female,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Female,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Male,Yes,Sun,Dinner,2,0.710345


total_bill   tip     sex smoker   day    time  size   tip_pct
smoker                                                                   
No     88        24.71  5.85    Male     No  Thur   Lunch     2  0.236746
       185       20.69  5.00    Male     No   Sun  Dinner     5  0.241663
       51        10.29  2.60  Female     No   Sun  Dinner     2  0.252672
       149        7.51  2.00    Male     No  Thur   Lunch     2  0.266312
       232       11.61  3.39    Male     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00  Female    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Male    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00  Female    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00  Female    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Male    Yes   Sun  Dinner     2  0.710345

total_bill    tip     sex smoker   day    time  size   tip_pct
smoker day                                                                     
No     Fri  94        22.75   3.25  Female     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00    Male     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00    Male     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00    Male     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Male    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Male    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Male    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00  Female    Yes  Thur   Lunch     4  0.115982

In [37]:
result = tips.groupby('smoker')['tip_pct'].describe()

display(result)

display(result.unstack('smoker'))

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

In [38]:
""" 
Quantile and Bucket Analysis: Random Sampling and Permutaion
Suppose you wanted to draw a random sample (with or without replacement)
from a large dataset for monte carlo simulation pusposes or some other application.
"""

# Hearts, Clubs, Diamonds

suits = ['H', 'S', 'C', 'D']

card_val = (list(range(1, 11)) + [10] * 3) * 4

base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'G']

cards = []

for suit in ['H', 'S', 'C', 'D']:
    cards.extend(str(num) + suit for num in base_names)


deck = pd.Series(card_val, index=cards)

display(deck)

# Now based on what was said before. drawing a hand of five cardfs from the dack can be written as

def draw(deck, n=5):
    return deck.sample(n)
    

display(draw(deck))

# If we wanted two random cards form each suit.
# Because the suit is the last character of each card name
# We can gruop based on this and use apply:

get_suit = lambda card: card[-1] # last latter is suit


display(deck.groupby(get_suit).apply(draw, n=2))

# Alternatively we could write:

display(deck.groupby(get_suit, group_keys=False).apply(draw, n=2))

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
GH     10
AS      1
2S      2
3S      3
4S      4
5S      5
6S      6
7S      7
8S      8
9S      9
10S    10
JS     10
KS     10
GS     10
AC      1
2C      2
3C      3
4C      4
5C      5
6C      6
7C      7
8C      8
9C      9
10C    10
JC     10
KC     10
GC     10
AD      1
2D      2
3D      3
4D      4
5D      5
6D      6
7D      7
8D      8
9D      9
10D    10
JD     10
KD     10
GD     10
dtype: int64

AH     1
9H     9
KH    10
AD     1
3H     3
dtype: int64

C  2C      2
   7C      7
D  5D      5
   8D      8
H  10H    10
   AH      1
S  10S    10
   KS     10
dtype: int64

9C      9
6C      6
8D      8
2D      2
4H      4
10H    10
6S      6
7S      7
dtype: int64

In [39]:
import pan

ModuleNotFoundError: No module named 'pan'

In [40]:
"""
# Example: Group Weighted Average and Correlation
Operations betwwen columns in a dataframe or two sesries,
such as a group weighted averege are possible
"""

df = pd.DataFrame({'category' : ['a', 'a', 'a', 'a',
                                'b', 'b', 'b', 'b'],
                  'data' : np.random.randn(8),
                  'weights': np.random.rand(8)})


display(df)


# The group weighted average by category would then be:

goruped = df.groupby('category')

get_wavga = lambda g: np.average(g['data'], weights=g['weights'])

goruped.apply(get_wavga)

,category,data,weights
0,a,-0.055361,0.259728
1,a,-0.153578,0.268413
2,a,0.512132,0.324183
3,a,-0.916284,0.029358
4,b,-0.526117,0.677544
5,b,0.945921,0.641292
6,b,0.564062,0.120186
7,b,-0.060684,0.986307


category
a    0.094732
b    0.106412
dtype: float64

In [51]:
# Financial Data example 

close_px = pd.read_csv('sp500_closefull.csv',
                      parse_dates=True,
                      infer_datetime_format=True,
                      index_col='Date')

display(close_px.info())

display(close_px[-4:])

# A task of interest might be to compute a df consisting of then yearly correlations of daily return
# (computed from percent changes) with SPX. As one way to do thism we first create a function that
# computes the pairwaise correlation of ech column with the 'SPX' column:

spx_corr = lambda x: x.corrwith(x['AMZN'])
spx_corr

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2270 entries, 2010-01-04 to 2018-12-31
Columns: 480 entries, CSCO to PPL
dtypes: float64(480)
memory usage: 8.3 MB


None

,CSCO,UAL,TROW,ISRG,PRGO,TPR,DVN,MRO,BA,VRTX,GILD,NLSN,EQIX,TIF,MDT,V,QRVO,A,MO,SWKS,MCHP,CDNS,WLTW,CHTR,EIX,BBY,WBA,HCA,AJG,DTE,C,T,CF,DISH,MGM,XL,HUM,CBOE,CFG,WU,APH,SYY,MSI,FCX,ADM,LH,STI,UTX,PKI,LNT,BAC,LNC,PSX,GPN,SRCL,PPG,IRM,IQV,ESS,NOV,NAVI,HAL,STZ,FLS,DXC,ADI,F,HOG,ADBE,TDG,ULTA,ARE,SYK,CB,TSN,FLR,PEP,PEG,LLY,COST,REG,NWS,LOW,MDLZ,FMC,XEL,AIZ,PDCO,CERN,MET,CHK,FTV,DLR,XRAY,SCG,FAST,TJX,SNA,MPC,D,CA,MRK,STX,NOC,BXP,KHC,IPG,UNP,ALLE,ABBV,ORCL,ECL,ETR,EBAY,SBUX,IR,AMT,INTU,DRE,JEC,CMA,PG,CAT,MCD,MNST,AMZN,INTC,PNR,GLW,BDX,KMI,PWR,APTV,EXR,HOLX,EXPD,GM,TXN,VRSK,SJM,TMO,OXY,RL,CCI,MMM,MOS,HSY,JNPR,DHI,ED,ES,ADSK,IP,EXPE,KO,PCAR,WDC,PYPL,NEE,UPS,FLIR,LEG,EMR,MSFT,ANSS,CTAS,UDR,RTN,WEC,AME,HP,IT,ACN,VRSN,EW,FL,CMG,AWK,COO,SHW,HPQ,AMAT,CCL,MLM,AVY,AAP,ATVI,EA,DE,SPG,AMD,MYL,KLAC,NDAQ,URI,WHR,PNC,KMX,WRK,BIIB,NVDA,CHRW,ROP,IDXX,EXC,HES,HD,ALB,VLO,AON,ZTS,FDX,DG,HIG,SIG,CMS,COL,CAG,INCY,SCHW,HSIC,AZO,AXP,HPE,DFS,SEE,HRL,SO,FRT,ZBH,CME,XOM,AMP,AMG,CVX,ETFC,CMCSA,PCG,CSRA,PNW,ICE,NFX,CTXS,TRIP,BEN,DISCK,UHS,EMN,SBAC,ROK,NRG,NSC,NKE,FIS,VTR,MAS,RF,MON,TAP,MAR,PX,XYL,CMI,FB,MTD,VAR,KR,PLD,IBM,USB,BSX,LKQ,FBHS,ITW,EOG,PVH,KMB,SPGI,NEM,WFC,ANDV,EL,GS,GD,CNP,PM,RE,MCO,CLX,CAH,CBS,HRB,DGX,AVB,DIS,GE,HII,ALL,ETN,ALGN,NFLX,LEN,BHF,FITB,GWW,NTRS,CVS,CTL,AOS,FE,ABC,JPM,ABT,CXO,OMC,COF,TSCO,PH,HST,AYI,JBHT,MAC,COP,DHR,COG,MAT,CNC,MCK,TXT,MTB,SPY,DISCA,AKAM,RMD,GOOGL,PAYX,ALK,DRI,ILMN,AAL,XLNX,MAA,MMC,EVHC,GT,FFIV,VNO,CINF,VMC,TWX,SRE,ORLY,IVZ,RCL,PXD,COTY,SNPS,GOOG,YUM,EQT,KSS,PFE,AIV,AVGO,DUK,REGN,CL,VFC,UA,VZ,JCI,ESRX,AMGN,TEL,ADP,AET,LB,STT,RRC,RSG,IFF,ANTM,GPS,BLL,QCOM,LYB,GIS,PHM,ROST,LUV,ALXN,XEC,MS,CPB,OKE,BK,SYF,CHD,SLG,MHK,INFO,DAL,APA,K,JWN,AFL,ADS,CSX,NI,PFG,NCLH,ZION,RJF,HBAN,UNH,PRU,GPC,FISV,WMB,EQR,PBCT,KSU,DVA,AIG,MA,HBI,HON,O,NWSA,AES,SLB,XRX,TGT,AAPL,MKC,WY,APD,GRMN,AEE,HLT,DLTR,HAS,WMT,NTAP,KIM,BAX,LMT,KEY,UNM,BMY,PSA,WYNN,RHI,EFX,NUE,PKG,NBL,CTSH,SWK,MU,TRV,L,AEP,CI,SNI,JNJ,WM,DOV,FTI,AGN,M,CRM,PGR,WAT,BWA,LRCX,NWL,UAA,BLK,PPL
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2018-12-26,42.470001,83.800003,89.650002,461.980011,40.700001,33.889999,22.709999,14.17,313.929993,161.839996,63.220001,23.530001,349.269989,77.800003,88.660004,130.229996,60.020000,65.540001,48.430000,66.709999,70.379997,42.59,148.860001,285.700012,56.200001,51.570000,67.720001,121.290001,71.309998,109.650002,51.439999,28.080000,42.599998,24.889999,23.200001,NaN,282.609985,94.120003,29.49,17.010000,78.849998,61.230000,112.169998,10.40,40.669998,124.540001,49.200001,106.010002,75.699997,41.680000,24.110001,50.619999,84.199997,101.440002,36.360001,99.269997,32.070000,111.190002,244.800003,26.040001,8.85,26.430000,163.419998,38.220001,52.130001,84.339996,7.85,33.810001,222.949997,332.489990,242.199997,113.970001,150.830002,125.029999,52.060001,32.310001,108.330002,50.709999,111.089996,198.630005,58.080002,11.36,91.040001,40.000000,63.694710,49.560001,87.070000,19.969999,51.560001,40.189999,438.0,65.699997,104.680000,37.189999,0.005,25.799999,43.840000,141.699997,58.349998,72.300003,59.099998,74.000000,37.910000,232.639999,112.739998,43.290001,20.35,136.020004,78.610001,89.040001,44.590000,142.830002,84.440002,27.850000,63.080002,20.219999,157.529999,192.800003,25.770000,57.860001,67.839996,90.089996,124.760002,174.029999,49.099998,1470.900024,46.189999,37.020000,29.180000,217.860001,15.64,29.780001,61.740002,90.309998,39.970001,65.970001,33.520000,92.980003,107.489998,94.779999,217.970001,60.599998,101.930000,107.629997,186.259995,29.309999,105.629997,26.600000,34.180000,76.089996,64.239998,125.930000,39.619999,114.199997,46.

<function __main__.<lambda>(x)>

In [53]:
# Next, we compute percent change on close_px using pct_change
rets = close_px.pct_change().dropna()
# Lasty, we group these percent changes by year, which can be extraxted from each row label
# With a oneline fucntion that returns the year attribute of each datetime label:

get_year = lambda x: x.year
display(get_year)


by_year = rets.groupby(get_year)
display(by_year)

by_year.apply(spx_corr)

# You could also compute inter-column correlations.
# Here we compute the annual correlation between Apple and Microsoft:

by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))

<function __main__.<lambda>(x)>

Date
2018    0.688011
dtype: float64

In [79]:
"""
10.4 Pivot Tables and Corss-Tabulation

A pivot table is a data aummarization tool frequently found in spreadsheet programs
and other data analysis software. It aggregates a table of data by one or more keys,
arranging the data in a rectagle with some of the group keys along the rows and some along the columns.
Pivot tables in Python are amde possible through the groupby fecility desbribed in this chapter 
combines with reshape operations utilizing hierarchinal indexing. Df has a pivot_table method,
and there is also a top-level pandas.pivot_table  fucntion. In addition to provpking a convenience interface

to groupby, pivot_table aggregation type
"""
fn_ratios = pd.read_csv('ratios_vals_money.csv',
                       parse_dates=True,
                       infer_datetime_format=True,
                       index_col=['Ticker', 'Year'])

finRatios = fn_ratios[:2500]

display(finRatios)

display(finRatios.pivot_table(index=[
    'Ticker', 
    'Research & Development', 
    'price_earnings_ratio'
]))

# This could have been produced with groupby directly.
# Now, to aggregate only tip_pct and size, and additionally group by time. I'll put smoker in the table
# columns and day in the rows

finRatios.pivot_table(index=['Ticker', 'Asets/Liabili'],
                     columns= 'Research & Development')

# We could augment this table to include partial totals by passing margins=True.
# this has the effect of adding all row and column labels, with corresponsing values being the group
# Statisticcs for allthe data within a single tier


# To use a different aggregation function, pass it to aggfuncs. For example 'count' or 
# len will give you a cross-tabulation (count or frequency

finRatios.pivot_table('Retained Earnings',
                      index=['Revenue','Net Cash from Operating Activities'],
                      columns='Year',
                     aggfunc=len, fill_value=0)

# pivot_table options page 321 

# Cros_tabulations: Crosstab




Close  Shares (Basic)  Shares (Diluted)   Revenue  Cost of Revenue  Gross Profit  Operating Expenses  Selling, General & Administrative  Research & Development  Depreciation & Amortization_x  Operating Income (Loss)  Non-Operating Income (Loss)  Interest Expense, Net  Pretax Income (Loss), Adj.  Abnormal Gains (Losses)  Pretax Income (Loss)  Income Tax (Expense) Benefit, Net  Income (Loss) from Continuing Operations  Net Extraordinary Gains (Losses)  Net Income  Net Income (Common)  Cash, Cash Equivalents & Short Term Investments  Accounts & Notes Receivable  Inventories  Total Current Assets  Property, Plant & Equipment, Net  Long Term Investments & Receivables  Other Long Term Assets  Total Noncurrent Assets  Total Assets  Payables & Accruals  Short Term Debt  Total Current Liabilities  Long Term Debt  Total Noncurrent Liabilities  Total Liabilities  Share Capital & Additional Paid-In Capital  Treasury Stock  Retained Earnings  Total Equity  \
Ticker Year                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
A      2015-01-01  39.602        0.172274          0.168954 -0.106620         0.127958     -0.036993           -0.000036                          -0.066345               -0.208035                       0.179789                -0.085018                     0.059474               0.155789                   -0.071408                 0.102475             -0.047871                           0.118054                                 -0.016521                         -0.157372   -0.030874            -0.029651                                         0.155972                    -0.043646     0.006434              0.055896                         -0.207525                            -0.073306               -0.023020                -0.130092     -0.076172            -0.187889        -0.171002                  -0.152952       -0.122725                     -0.125281          -0.147966                                    0.928680       -1.075697           0.209957      0.051456   
       2016-01-01  43.370        0.162817          0.160931 -0.099310         0.127476     -0.015615           -0.013266                          -0.084542               -0.206883                       0.179789                -0.055833                     0.031520               0.149547                   -0.049564                 0.102475             -0.026186                           0.072776                                 -0.006239                         -0.030193   -0.004714            -0.003491                                         0.205335                    -0.037257     0.002733              0.051442                         -0.204437                            -0.065501               -0.001508                -0.114804     -0.066475            -0.192370        -0.171002                  -0.156650       -0.093715                     -0.107180          -0.136744                                    0.944572       -1.128644           0.243927      0.057234   
       2017-01-01  59.330        0.157412          0.156919 -0.087275         0.123979      0.013437

Abnormal Gains (Losses)  Accounts & Notes Receivable  Asets/Liabili  Cash from (Repayment of) Debt  Cash from (Repurchase of) Equity  Cash, Cash Equivalents & Short Term Investments  Change in Accounts Payable  Change in Accounts Receivable  Change in Fixed Assets & Intangibles  Change in Inventories  Change in Other  Change in Working Capital   Close  Cost of Revenue  Depreciation & Amortization_x  Depreciation & Amortization_y  Dividends Paid  Earnings_per_share_ratio  Gross Profit  Income (Loss) from Continuing Operations  Income Tax (Expense) Benefit, Net  Interest Expense, Net  Interest_coverage_ratio  Inventories  Long Term Debt  Long Term Investments & Receivables  Net Cash from Acquisitions & Divestitures  Net Cash from Financing Activities  Net Cash from Investing Activities  Net Cash from Operating Activities  Net Change in Cash  Net Change in Long Term Investment  Net Extraordinary Gains (Losses)  Net Income  \
Ticker Research & Development price_earnings_ratio                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         
A      -0.293300               42.994261                           0.102475                     0.039162      -0.587271                       0.218347                         -0.300168                                         0.048789                    0.119691                      -0.470095                              0.149423              -0.089807        -0.029453                  -0.096397  75.212         0.106193                       0.179789                      -0.097299        0.012263                  1.749350      0.067703                                  0.254674                           0.337649               0.221335                 0.209948     0.070285       -0.106880                            -0.070758                                  -0.568105                           -0.012263                           -0.338419                            0.029698           -0.666750                            0.014432                         -0.030193    0.256460   
       -0.271408              -155.694957                          0.102475                    -0.000198      -0.540063                      -0.188147                         -0.100412                                         0.198086                    0.180338                      -0.264968                              0.136753              -0.269394         0.043320                  -0.040409  67.169         0.114091                       0.179789                      -0.118313        0.029059                 -0.431414      0.051533                                 -0.068789                          -0.547523               0.224456                 0.188853     0.051315       -0.105948                            -0.076173                                  -0.142672                           -0.200712                           -0.028478                            0.049273           -0.342570                            0.014432                         -0.030193   -0.067327   
       -0.218405               103.204712                          0.102475                    -0.01

Year                                           2015-01-01  2016-01-01  2017-01-01  2018-01-01  2019-01-01
Revenue    Net Cash from Operating Activities                                                            
-0.286611  -0.329583                                    0           0           1           0           0
           -0.320364                                    0           0           0           1           0
           -0.312932                                    0           0           0           1           0
           -0.306378                                    0           1           0           0           0
           -0.298866                                    0           0           0           1           0
...                                                   ...         ...         ...         ...         ...
 11.062673  11.201275                                   0           0           0           0           1
 11.158954  3.537536                                    0           0           0           0           1
 11.310416  20.307952                                   0           0           0           0           1
 11.552053  22.693472                                   0           0           0           1           0
 12.217411  11.149964                                   0           0           0           0           1

[2499 rows x 5 columns]